In [2]:
!pip install kaggle-environments -U > /dev/null


In [1]:
from kaggle_environments import make

Loading environment football failed: No module named 'gfootball'


In [2]:
env = make("lux_ai_2021", configuration={"seed": 562124210, "loglevel": 2, "annotations": True}, debug=True)

In [21]:
%%time
steps = env.run(["simple_agent", "simple_agent"])

CPU times: user 3.28 s, sys: 86.2 ms, total: 3.37 s
Wall time: 3.47 s


In [18]:
## we should append our result
import sys
sys.path.insert(0, "/workspace/worskpace/Lux-Design-2021/kits/python/simple")


In [129]:
from lux.game import Game
from lux.game_map import Cell, RESOURCE_TYPES, Position
from lux.constants import Constants
from lux.game_constants import GAME_CONSTANTS
from lux import annotate
import math
import sys

import numpy as np
import pandas as pd

from numpy.random import choice

In [43]:
# we declare this global game_state object so that state persists across turns so we do not need to reinitialize it all the time
game_state = None
def agent(observation, configuration):
    global game_state

    ### Do not edit ###
    if observation["step"] == 0:
        game_state = Game()
        game_state._initialize(observation["updates"])
        game_state._update(observation["updates"][2:])
        game_state.id = observation.player
    else:
        game_state._update(observation["updates"])

    actions = []

    ### AI Code goes down here! ### 
    player = game_state.players[observation.player]
    opponent = game_state.players[(observation.player + 1) % 2]
    width, height = game_state.map.width, game_state.map.height

    # add debug statements like so!
    if game_state.turn == 0:
        print("Agent is running!", file=sys.stderr)
        actions.append(annotate.circle(0, 0))
    return actions

In [44]:
# this snippet finds all resources stored on the map and puts them into a list so we can search over them
def find_resources(game_state):
    resource_tiles: list[Cell] = []
    width, height = game_state.map_width, game_state.map_height
    for y in range(height):
        for x in range(width):
            cell = game_state.map.get_cell(x, y)
            if cell.has_resource():
                resource_tiles.append(cell)
    return resource_tiles

# the next snippet finds the closest resources that we can mine given position on a map
def find_closest_resources(pos, player, resource_tiles):
    closest_dist = math.inf
    closest_resource_tile = None
    for resource_tile in resource_tiles:
        # we skip over resources that we can't mine due to not having researched them
        if resource_tile.resource.type == Constants.RESOURCE_TYPES.COAL and not player.researched_coal(): continue
        if resource_tile.resource.type == Constants.RESOURCE_TYPES.URANIUM and not player.researched_uranium(): continue
        dist = resource_tile.pos.distance_to(pos)
        if dist < closest_dist:
            closest_dist = dist
            closest_resource_tile = resource_tile
    return closest_resource_tile

In [ ]:
# lets look at some of the resources found
resource_tiles = find_resources(game_state)
cell = resource_tiles[0]
print("Cell at", cell.pos, "has")
print(cell.resource.type, cell.resource.amount)

In [46]:
env = make("lux_ai_2021", configuration={"seed": 562124210, "loglevel": 2, "annotations": True}, debug=True)
steps = env.run([agent, "simple_agent"])

Agent is running!


In [114]:
def retrieve_information_from_game_state(game_state, player_id=0):

    """
    Retrieve the map in a tensor format :
    map => (h, w, 12) tensor

    0 channel is the cities information
    1-2 channels is the units information
    3-5 is the ressources information

    """

    map = game_state.map

    w = map.width
    h = map.height

    array_cities = np.zeros((h, w, 1))
    array_units = np.zeros((h, w, 2)) # 2 channels for the units types

    array_ressource = np.zeros((h, w, 3)) ## 3 channels for all resosurces

    # Here we add the ressource on the map
    for y in range(w):
            for idx in range(h):
                cell = map.map[y][idx]
                if cell.has_resource():

                    if cell.resource.type == Constants.RESOURCE_TYPES.WOOD:
                        array_ressource[y, idx, 0] = 1
                    if cell.resource.type == Constants.RESOURCE_TYPES.COAL:
                        array_ressource[y, idx, 1] = 1
                    if cell.resource.type == Constants.RESOURCE_TYPES.URANIUM:
                        array_ressource[y, idx, 2] = 1

    player = game_state.players[player_id]

    # Here we add information about units and cities
    for unit in player.units:
        # if the unit is a worker (can mine resources) and can perform an action this turn
        if unit.is_worker() and unit.can_act():
            pos = unit.pos
            array_units[pos.x, pos.y, 0] = 1
            array_units[pos.x, pos.y, 1] = unit.get_cargo_space_left()/70
            
    for city in player.cities:
        city_class = player.cities[city]
        for citytile in city_class.citytiles:
            pos = citytile.pos
            array_cities[pos.x, pos.y, 0] = 1 

    player = game_state.players[(player_id + 1) % 2]

    # Here we add information about units and cities
    for unit in player.units:
        # if the unit is a worker (can mine resources) and can perform an action this turn
        if unit.is_worker() and unit.can_act():
            pos = unit.pos
            array_units[pos.x, pos.y, 0] = -1

    # citytiles computation
    for city in player.cities:
        city_class = player.cities[city]
        for citytile in city_class.citytiles:
            pos = citytile.pos
            array_cities[pos.x, pos.y, 0] = -1 

    return np.concatenate((array_cities, array_units, array_ressource), axis=-1)

In [ ]:
retrieve_information_from_game_state(game_state)

In [ ]:
def compute_action(action_array_cities, actions_array_units, game_state, player_id=0, state_array):
    """
    In input of the model we take 
    
    action_array_cities = (w, h, c) where c is the number of action that the city can take (reseach, build worker of do nothing)
    those are probability values so for each pixel we have we can manage what action will be taken with the mask value
    c is 3 here
    
    actions_array_units = (w, h, c') where c' is the number of action that our units can take
    c' is 5 for direction + 1 for build city action + ...
    """

    player = game_state.players[(player_id + 1) % 2]

    actions =[]
    
    action_list_units = [Constants.DIRECTIONS.CENTER, Constants.DIRECTIONS.EAST,
                         Constants.DIRECTIONS.NORTH, Constants.DIRECTIONS.SOUTH, Constants.DIRECTIONS.WEST]

    ## this part manage the units action
    for unit in player.units:
        # if the unit is a worker (can mine resources) and can perform an action this turn
        if unit.is_worker() and unit.can_act():

            pos = unit.pos

            # we retrieve the position from the image
            p_actions = actions_array_units[pos.x, pos.y, :]

            # we mask the actions we can't do
            if unit.can_build() == False:
                p_actions[-1] = 0

            # check direction too ... TODO

            # renormalize action 
            p_actions = p_actions / np.sum(p_actions)

            # select action according to the probability
            tmp_action_index = choice(np.arange(6), 1, p=p_actions)

            if tmp_action_index == 5:
                ## building city
                tmp_action = unit.build_city()
                actions.append(tmp_action)
            else:
                # move action
                tmp_action = unit.move(action_list_units[tmp_action_index])
                actions.append(tmp_action)

    ## this part manage the city part
    for city in player.cities:
        city_class = player.cities[city]
        for citytile in city_class.citytiles:

            ## we retrieve the citytile position
            pos = citytile.pos
            p_actions = action_array_cities[pos.x, pos.y, :]

            if citytile.can_act() == False:
                p_actions[-1] = 0
                p_actions[-2] = 0

            # renormalize action 
            p_actions = p_actions / np.sum(p_actions)
            
            tmp_action_index = choice(np.arange(3), 1, p=p_actions)
            
            if tmp_action_index == 0:
                # we do nothing
                pass
            elif tmp_action_index == 1:
                tmp_action = citytile.research()
                actions.append(tmp_action)
            elif tmp_action_index == 2:
                tmp_action = citytile.build_worker()
                actions.append(tmp_action)
                
    return actions


In [84]:
game_state.map.map[0][0].has_resource()

False

In [66]:
game_state.players[0].units[0].pos.x

3